# Notebook for make profiling of di-f Correlation experiments

## Experiment name: mxretailsalary1

## Experiment General Data
### Team roles:
* PipeMaster: jag.pascoe
* BizEngineer: 
* DataEngineer:
* MLEngineer:
* SWEngineer:

### Description (Use case):
Predict salary per day estimation to be obtained for working in retail sector in any state of Mexico.
Supposing you are looking for being hired in a Retail Business in any of Mexico's state you want to. You want to predict which would be the base salary per day you might get as attendant of that retail business. This salary not include any commision, tax, or any other concept.

### Type of experiment: Correlation
### Independent Variables (inputs):
1) State of Mexico where you are supposing to get hired (CAT). 
2) How many employees (including yourself) work in that particular business now (NUMBER)
3) How much sales per day in average, you estimate you will provide to that business in pesos (FLOAT)

### Dependent Variables (outputs):
1) Estimated base salary per day (FLOAT)

## Experiment preparation, imports and config.yaml

In [1]:
%load_ext autoreload
%autoreload 2
# The %load_ext autoreload and %autoreload 2 magic commands are used to automatically 
# reload modules when they are changed. This can be useful when you are developing code 
# in an interactive environment, as it allows you to see the changes you make to your modules 
# without having to restart the kernel.
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
import pandas as pd
import numpy as np
import ydata_profiling as yp
import os

# for global initialization: NOT RECOMMENDED
#initialize(version_base=None, config_path="../src/conf")
#compose(config_name='config')

with initialize(version_base=None, config_path="../src/conf"):
    cfg = compose(config_name='config')
    print(cfg)

/home/jagpascoe/democlient-sklearn/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/jagpascoe/democlient-sklearn/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


{'general_ml': {'seed': 123, 'encoding': 'iso-8859-1', 'cloud': 'AWS'}, 'paths': {'project_dir': '...', 'raw_data': '${hydra:runtime.cwd}/data/raw', 'interim_data': '${hydra:runtime.cwd}/data/interim', 'processed_data': '${hydra:runtime.cwd}/data/processed', 'reports': '${hydra:runtime.cwd}/reports'}, 'cloud_paths': {'bucket_path': 'dif-b-democlient-sklearn', 'experiment_path': '${cloud_paths.bucket_path}/mxretailsalary1', 'mlflow_path': '${cloud_paths.experiment_path}/mlflow', 'reports_path': '${cloud_paths.experiment_path}/reports', 'rawdata_path': '${cloud_paths.experiment_path}/raw-data', 'dvc_path': '${cloud_paths.experiment_path}/dvc-store'}, 'file_names': {'raw_file': 'raw-data.csv', 'data_file': 'datafile.csv', 'train_features': 'train_features.csv', 'train_labels': 'train_labels.csv', 'validation_features': 'valid_features.csv', 'validation_labels': 'valid_labels.csv', 'test_features': 'test_features.csv', 'test_labels': 'test_labels.csv', 'data_profiling_report': 'data_profil

In [2]:
#reading raw-data

raw_file = pd.read_csv(os.path.join('../data/raw', cfg.file_names.raw_file), 
                   #encoding=cfg.general_ml.encoding,
                   )
raw_file.head()

,state,municipio,businesses,employees,payroll,expenditures,income,payroll_employee_day,profits_biz_day,sales_employee_day,employees_unit
0,Ags,Aguascalientes,11402,120923,4997.813674,129044.643300,159695.334000,114.807074,6249.604612,3668.434872,10.6054201
1,Ags,Asientos,231,1647,29.092255,524.807734,621.055144,49.066071,807.5415528,1047.451838,7.12987013
2,Ags,Calvillo,591,4605,99.342787,1743.278407,2246.247441,59.924470,1897.09648,1354.956835,7.791878173
3,Ags,Cosio,104,468,4.577495,101.941048,136.364948,27.169366,797.1796419,809.383597,4.5
4,Ags,El Llano,104,860,14.802274,342.630524,427.291666,47.810963,1865.888567,1380.141041,8.269230769


In [3]:
#cutting dataset for this experiment
data=raw_file[['state',
        'businesses',
        'employees',
        'payroll',
        'income']]
data

,state,businesses,employees,payroll,income
0,Ags,11402,120923,4997.813674,159695.334000
1,Ags,231,1647,29.092255,621.055144
2,Ags,591,4605,99.342787,2246.247441
3,Ags,104,468,4.577495,136.364948
4,Ags,104,860,14.802274,427.291666
...,...,...,...,...,...
2482,Zacatecas,147,785,2.811121,160.114046
2483,Zacatecas,149,875,4.875017,289.174975
2484,Zacatecas,96,604,7.800028,219.851736
2485,Zacatecas,339,2043,34.189010,1282.438977


In [4]:
#Taking a sample of the data, reducing from 2487 records to 750
data=data.sort_values(by='businesses',ascending=False)
data = data.head(750)
data = data.sample(frac=1).reset_index(drop=True)
data.groupby('state').count() #to show data distribution by state

,businesses,employees,payroll,income
state,,,,
Ags,6,6,6,6
BC,6,6,6,6
BCS,5,5,5,5
CDMX,17,17,17,17
Campeche,8,8,8,8
Chiapas,51,51,51,51
Chihuahua,10,10,10,10
Coahuila,15,15,15,15
Colima,7,7,7,7


## Data preparation and homologation for EDA

In [5]:
#reviewing data types
data.dtypes

state          object
businesses      int64
employees       int64
payroll       float64
income        float64
dtype: object

In [6]:
# Finding number of records with 0, NaN, or empty values
mask=data.apply(lambda x: any([val == 0 or pd.isna(val) or val == '' for val in x]), axis=1)
mask.sum()



0

In [7]:
#finding recosrds with nan 
data.isna().sum()

state         0
businesses    0
employees     0
payroll       0
income        0
dtype: int64

In [8]:
#correcting nan in Payroll by substitute them with the payrrol mean by estado.


# Compute the average by category
payroll_average = data.groupby('state')['payroll'].transform('mean')
employees_average = data.groupby('state')['employees'].transform('mean')


# Replace NaN values with the corresponding category average
payroll2 = data['payroll'].fillna(data['employees']*payroll_average/employees_average)
data['payroll']=payroll2

# Display the modified dataframe
print(data[mask])


Empty DataFrame
Columns: [state, businesses, employees, payroll, income]
Index: []


In [9]:
# check records with zeros
data[data['employees']==0]
data[data['businesses']==0]

,state,businesses,employees,payroll,income


In [10]:
# Replace cero values with other value
data.loc[data['businesses'] == 0, 'businesses'] = 1.0 #in this case the better value to filna is 1 business at least
data.head()


,state,businesses,employees,payroll,income
0,Nuevo Leon,14061,248258,8289.068764,338631.196600
1,Oaxaca,905,3905,15.752988,662.353238
2,Zacatecas,349,2988,43.555710,1758.804061
3,Mexico,16237,113409,2218.805161,136972.510900
4,Tlaxcala,443,1979,25.286211,463.621475


In [11]:
# create new fields 

data['income_employee_day']=data['income']*1000000/data['employees']/360
data['employees_business'] = (data['employees']/data['businesses']+0.5).astype(int)
data['salary_employee_day']=data['payroll']*1000000/data['employees']/360

#droping innecesary fields
data=data.drop(['employees', 'income','businesses','payroll'],axis=1)

# correcting data types and renaming the fields to be more accurate
data['state']=data['state'].astype('category')

data.head()

,state,income_employee_day,employees_business,salary_employee_day
0,Nuevo Leon,3788.970397,18,92.747025
1,Oaxaca,471.157517,4,11.205710
2,Zacatecas,1635.062529,9,40.491327
3,Mexico,3354.929476,7,54.346195
4,Tlaxcala,650.751607,4,35.492408


## EDA Engineering Data Analisys

### EDA with pandas profiling, and saving report in ./reports

In [ ]:
#running the profiler to review data, saving the report on ./reports directory

ProfileReport = yp.ProfileReport(data, title=str(cfg.file_names.data_profiling_report), explorative=True)
ProfileReport.to_file(os.path.join('../reports', cfg.file_names.data_profiling_report), 
                      #encoding=cfg.general_ml.encoding
                     )

ProfileReport

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

### EDA at detail with pandas, matplotlib and seaborn

In [ ]:
# To be developed

## ML profiling with Pycaret

In [ ]:
#Choose the Ml model to be applied, among: regression, Classifications, time_series, Clustering, NLP
from pycaret.regression import *
from pycaret import version_
version_

In [ ]:
#to show docstring of setup() function
?setup

### Model to find
Setup the model with characteristics needed to get the key model.
The setup() function initializes the experiment in PyCaret and creates the transformation pipeline based on all the parameters passed in the function.

In [ ]:
model_to_find =   setup(data = data, #see above 
                        #log_experiment = True,
                        #experiment_name = f'{cfg.general_ml.client}-{cfg.general_ml.project}-{cfg.general_ml.experiment}',
                        #target = cfg.data_fields.label, # get the target label from cfg
                        target='salary_employee_day',
                        #train_size=0.6, #default = 0.7
                        session_id=cfg.general_ml.seed, # get the seed from config
                        #train_size = 1.0-float(cfg.data_pipeline.data_transform_params.percent_valid), #get %valid from cfg
                        transformation=True, 
                        #transformation_method='quantile',
                        #fix_imbalance = True, #8:2
                        normalize=True,
                        normalize_method="minmax",
                        #polynomial_features=True,
                        #polynomial_degree = 5,
                        max_encoding_ohe =32, #default=25,
                        #remove_multicollinearity=True,
                        #categorical_features=['state'],
                        #remove_outliers=True,
                        #outliers_threshold=0.075, #default=0.05
                        )

In [ ]:
#with this model_to_find, let's see how was transformed the dataset 
get_config('dataset_transformed')

### Getting the best model 
From all the possibles using compare_models() function.
This function trains and evaluates the performance of all estimators available in the model library using cross-validation. The output of this function is a scoring grid with average cross-validated scores. Metrics evaluated during CV can be accessed using the get_metrics function. Custom metrics can be added or removed using add_metric and remove_metric function.

In [ ]:
best_model=compare_models( sort='R2', #from which metric you will choose the best model
                          #include=['list', 'of', 'models'], #list of models to be included, comment if all
                          #n_select= 3, #if you want you can get the Top N models instead of just one model. 
                          cross_validation=True, #If you don't want to evaluate models using cross-validation
                          )

In [ ]:
best_model.get_params()



### Selected Model
Selected_model is the best model choosen, you may want to use another kfolds param to get this model.

This function trains and evaluates the performance of a given estimator using cross-validation. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the get_metrics function. Custom metrics can be added or removed using add_metric and remove_metric function. All the available models can be accessed using the models function.

In [ ]:
models() #to show available models in this library

In [ ]:
# And create the selected model in ten kfolds
selected_model = create_model(best_model,
                              #'gbr', 
                              fold=5, #default = None. Controls cross-validation. Integer:'n_splits'
                              cross_validation=True,
                              return_train_score=False, #To see the performance metrics on the training set by fold 
                              )

In [ ]:
print(selected_model, selected_model.get_params())

In [ ]:
#train models in a loop
selected_models  = [create_model('huber', alpha = i) for i in [0.01, 0.001, 0.0001, 0.00001]]
#you will see a table for each option:

In [ ]:
# You can also write your own class with fit and predict function. 
# PyCaret will be compatible with that. Here is a simple example

"""
# load dataset 
from pycaret.datasets #import get_data 
insurance= get_data('insurance') 

# init setup
from pycaret.regression import * 
reg1 = setup(data = insurance, target = 'charges')

# create custom estimator
import numpy as np
from sklearn.base import BaseEstimator
class MyOwnModel(BaseEstimator):
    
    def __init__(self):
        self.mean = 0
        
    def fit(self, X, y):
        self.mean = y.mean()
        return self
    
    def predict(self, X):
        return np.array(X.shape[0]*[self.mean])
        
# create an instance
my_own_model = MyOwnModel()

# train model
my_model_trained = create_model(my_own_model)
"""

### Tunned Model
This function tunes the hyperparameters of the model. The output of this function is a scoring grid with cross-validated scores by fold. The best model is selected based on the metric defined in optimize parameter. Metrics evaluated during cross-validation can be accessed using the get_metrics function. Custom metrics can be added or removed using add_metric and remove_metric function.

The tuning grid for hyperparameters is already defined by PyCaret for all the models in the library. However, if you wish you can define your own search space by passing a custom grid using custom_grid parameter. 


In [ ]:
# Now lets find the best hyperparameters for the selected model 

tuned_model = tune_model(selected_model, 
                         optimize = 'r2', #which metric to optimize
                         n_iter=10,
                         
                         choose_better=True, #will always return a better performing model meaning that if 
                                             #hyperparameter tuning doesn't improve the performance, 
                                             #it will return the input model.
                         
                         custom_grid={ #comment if need full search:
                                         'alpha': [0.01, 0.001, 0.0001], 
                                         'epsilon': [.95, 1.05, 1.25, 1.35],
                                      }
                        )

#NOTE: choose_better doesn't affect the scoring grid that is displayed on the screen. 
#The scoring grid will always present the performance of the best model as selected by the tuner, 
#regardless of the fact that output performance < input performance.



In [ ]:
#PyCaret integrates seamlessly with many different libraries for hyperparameter tuning. 
#This gives you access to many different types of search algorithms including random, bayesian, optuna, TPE, 
#and a few others. All of this just by changing a parameter. By default, PyCaret using RandomGridSearch 
#from the sklearn and you can change that by using search_library and search_algorithm parameter in
#the tune_model function
"""
# load dataset
from pycaret.datasets import get_data 
boston = get_data('boston') 

# init setup
from pycaret.regression import * 
reg1 = setup(boston, target = 'medv')

# train model
dt = create_model('dt')

# tune model sklearn
tune_model(dt)

# tune model optuna
tune_model(dt, search_library = 'optuna')

# tune model scikit-optimize
tune_model(dt, search_library = 'scikit-optimize')

# tune model tune-sklearn
tune_model(dt, search_library = 'tune-sklearn', search_algorithm = 'hyperopt')
"""

### Essemble models
This function ensembles a given estimator. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the get_metrics function. Custom metrics can be added or removed using add_metric and remove_metric function.+

![image.png](attachment:image.png)




** Bagging: 
Also known as Bootstrap aggregating, is a machine learning ensemble meta-algorithm designed to improve the stability and accuracy of machine learning algorithms used in statistical classification and regression. It also reduces variance and helps to avoid overfitting. Although it is usually applied to decision tree methods, it can be used with any type of method. Bagging is a special case of the model averaging approach.

In [ ]:
bagging_model = ensemble_model(tuned_model, 
                               n_estimators=100, #By default, PyCaret uses 10 estimators
                               choose_better=True, #When set to True it will always return a better performing model 
                                                   #meaning that if hyperparameter tuning doesn't improve the performance, 
                                                   #it will return the input model
                               method = 'Bagging')

In [ ]:
print(bagging_model)



** Boosting: 
Is an ensemble meta-algorithm for primarily reducing bias and variance in supervised learning. Boosting is in the family of machine learning algorithms that convert weak learners to strong ones. A weak learner is defined to be a classifier that is only slightly correlated with the true classification (it can label examples better than random guessing). In contrast, a strong learner is a classifier that is arbitrarily well-correlated with the true classification.

In [ ]:
boosted_model = ensemble_model(tuned_model, 
                               n_estimators=100, #By default, PyCaret uses 10 estimators
                               choose_better=True, #When set to True it will always return a better performing model 
                                                   #meaning that if hyperparameter tuning doesn't improve the performance, 
                                                   #it will return the input model
                               method = 'Boosting')

In [ ]:
print(boosted_model)

### Blend model
This function trains a Soft Voting / Majority Rule classifier for select models passed in the estimator_list parameter. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the get_metrics function. Custom metrics can be added or removed using add_metric and remove_metric function

In [ ]:
top_best_models = ['huber','br','lr','gbr']
top_models=[]
for i,model in enumerate(top_best_models):
    print(f'{i+1} of {len(top_best_models)}: model: {model}')
    top_models.append(create_model(model))

print(f'Blend for {i+1} Models:')
blend_model = blend_models(top_models,
                          fold = 5, #evaluation is done using fold cross-validation.
                          weights=[0.25, 0.25, 0.40, 0.1], #weights to be given to each input model.
                          )

print(blend_model)

In [ ]:
# or a shorcut using compare_models() function
blend_model_shortcut = blend_models(compare_models(n_select=4)
                                   weights=[0.25, 0.25, 0.40, 0.1], #weights to be given to each input model.
                                   )

print(blend_model_shortcut)

### Stack model
This function trains a meta-model over select estimators passed in the estimator_list parameter. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the get_metrics function. Custom metrics can be added or removed using add_metric and remove_metric function.

In [ ]:
top_best_models = ['huber','br','lr','gbr']
top_models=[]
for i,model in enumerate(top_best_models):
    print(f'{i+1} of {len(top_best_models)}: model: {model}')
    top_models.append(create_model(model))

print(f'Stack for {i+1} Models:')
stack_model = stack_models(top_models,
                          fold = 5, #evaluation is done using fold cross-validation.
                          )

print(stack_model)

In [ ]:
# or a shorcut using compare_models() function
stack_model_shortcut =stack_models(compare_models(n_select=4)
                                   )

print(blend_model_shortcut)

### Model Evaluation


** Plot the model (simple function)
This function analyzes the performance of a trained model on the hold-out set. It may require re-training the model in certain cases

In [ ]:
plot_model(tuned_model, 
           plot='error',
           #save=True,
          )

In [ ]:
#https://pycaret.gitbook.io/docs/get-started/functions/analyze#plot_model
    

** Evaluate Model with evaluate_model()

The evaluate_model() displays a user interface for analyzing the performance of a trained model. 
It calls the plot_model().


In [ ]:
# evaluate the model

evaluate_model(tuned_model)



In [ ]:
# Only for classification
#interpret_model(tuned_model)

### interpret model

** interpret_model
This function analyzes the predictions generated from a trained model. Most plots in this function are implemented based on the SHAP (Shapley Additive exPlanations). 

In [ ]:
#interpret_model(tuned_model) # this functions is only for trees

** dashboard
The dashboard function generates the interactive dashboard for a trained model. The dashboard is implemented using ExplainerDashboard 

In [ ]:
dashboard(tuned_model)

### deep check
this function runs a full suite check over a trained model using the deepcheck library

In [ ]:
deep_check(tuned_model)

### eda - somed day will work!

In [133]:
#eda(display_format = 'svg')

### Predict
This function generates the label using a trained model.  When data is None, it predicts label and score on the holdout set. 

In [134]:
predict_model(tuned_model)

In [769]:
# Finally trains a the model on the entire dataset including the hold-out set.

final_model = finalize_model(tuned_model)
final_model




Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['income_employee_day',
                                             'employees_business'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['state'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('rest_encoding',
                 TransformerWrapper(include=['state'],
                                    tra...=TargetEncoder(cols=['state'],
                                                              handle_missing='return_nan'))),
                ('remove_outliers',
                 TransformerWrapper(transformer=RemoveOutliers(random_state=123))),
                ('transformation',
                 TransformerWrapper(transformer=PowerTransformer(standardize=False))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('actual_estimator', LGBMRegressor(random_state=123))])

In [770]:
get_config('X_transformed')

,state,income_employee_day,employees_business
63,0.201075,1.425386,0.687166
34,0.979646,-0.655274,-0.201409
33,1.092863,-1.259237,-1.777388
583,0.182101,2.810290,0.687166
457,-0.484366,1.482044,1.475067
...,...,...,...
617,-1.340800,-1.861335,-1.777388
226,-0.086717,-1.053872,-0.201409
26,1.238457,-1.687961,-0.861279
348,1.092863,1.158161,-0.861279


In [771]:
# save the model

#save_model(final_model, os.path.join(cfg.paths.models, cfg.file_names.ml_profiling_model))
